# Ejercicio 1

Por iteración, vamos a hacer lo siguiente:

1. Se generará una población de vectores 

1. **Primera iteración:** Se genera una población de $2k$ individuos aleatoria (cada individuo es un vector de números reales).

2. Se evalúan los individuos mediante la **función de aptitud** y se les asigna un 'pedazo del pastel' en la ruleta (proporcional a 'que tan buenos' son): 
    * Tendremos una `sumaAptitudes` **($F$)** que nos va a determinar 'el tamaño del pastel completo'. Éste es la suma de todas las aptitudes ($eval(v_i)$).
    * Para cada individuo $v_i$, tendremos una **probabilidad de selección asociada** $p_i$. Ésta está dada por: $\frac{eval(v_i)}{F}$.
    * El 'pedazo de pastel' que le corresponde a un individuo es su probabilidad de ser seleccionado para la siguiente generación. Ésta depende de su aptitud entre la suma de aptitudes. Vamos a considerar una probabilidad acumulativa para facilitar la implementación.
        * Por 'probabilidad acumulativa' nos referimos a:
        
            Si tienes una lista de individuos con sus probabilidades $p_i$ las probabilidades acumulativas $q_i$ crean una escala o 'intervalos' en los que se ubica cada individuo.

            Por ejemplo si el **Individuo 1** tiene una probabilidad de 0.1, y el **Individuo 2** tiene una probabilidad de 0.2, entonces: 
            
            $q_1 = 0.1$ (el intervalo del **Individuo 1** es de 0 a 0.1) 
            
            $q_2 = 0.1 + 0.2 = 0.3$ (el intervalo del **Individuo 2** es de 0.1 a 0.3)

3. Con probabilidad uniforme (pensando en que estamos lanzando un dardo a la ruleta) generamos un número $r \in [ 0,1 ]$. Si $r < q_i$ (si el dardo cae en el pedazo de pastel correspondiente al individuo $v_i$), el individuo $v_i$ es elegido como el primer padre. 
    
4. Aquí viene la parte de generación de crías.     
    * Para evitar la rápida pérdida de diversidad, no vamos a permitir un RadagonXMarika (no se vale generar hijos con un sólo padre). Seguro esto se puede hacer usando algún `if`.
    * Vamos a generar $2k - 1$ hijos (pues en cada generación, la idea es quedarnos con el mejor y matar a los demás).
    * En este paso, vamos a hacer un paso intermedio donde se codifica a los padres como vectores binarios y por cromosoma vamos a aplicar el operador de reproducción correspondiente.
        * Nuestra implementación es algo del estilo: `[[0,1,1,0,...], ..., [0,1,1,1,...]]`. Por 'cromosoma' nos referimos a cada uno de los vectores de 0s y 1s (pues en nuestra implementación, los vectores de números reales pasan a ser vectores de vectores de 0s y 1s).

5. Antes de actualizar la nueva generación, vamos a pasar la lista de los $2k - 1$ hijos por el operador de mutación flip (en principio 1-flip). **ESTO VISÍTALO MÁS ADELANTE**.

6. Tenemos una nueva población de $2k$ individuos a la que le vamos a repetir el proceso.



Tenemos un total de 5 funciones de prueba y cada una tiene intervalos de búsqueda diferentes. 

In [3]:
'''Función que nos genera una población inicial de 2k individuos y 
   recibe como parámetros: La 'k' y el hipercubo al que pertenecen 
   los vectores (o sea, la dimensión 'n' y el intervalo [a,b] al que
   pertenecen cada una de las entradas de los vectores).'''

import numpy as np

def Generar_Poblacion(k, n, intervalo):
    
   a, b = intervalo

   # Se genera una matriz de tamaño (2k, n) con valorios aleatorios
   # uniformes (o sea, una población inicial de 2k individuos donde 
   # cada uno vive en Rn):
   poblacionMatriz = np.random.uniform(a,b, size=(2*k, n))

   # Para facilitarnos la vida, convertimos a la matriz en una lista
   # de listas:
   poblacionLista = poblacionMatriz.tolist()

   return poblacionLista


In [4]:
'''Veamos un ejemplo de implementación'''

n = 3 # Dimensión en la que están los vectores
k = 2 # Queremos 4 individuos.
intervalo = (-5, 5) # Para efectos del ejemplo, usaremos el intervalo de
                    # la función Rastrigin.

poblacionInicial = Generar_Poblacion(k, n, intervalo)

print(f"Problación generada:\n {poblacionInicial}")
print(f"De {len(poblacionInicial)} individuos.")


Problación generada:
 [[-3.7994162112155996, -2.987784563046424, 1.4865941259827675], [-3.8853196605015006, -3.410093325744674, -0.7043442609623076], [4.237166788184384, 4.530236467502197, 3.012692100725161], [3.681870169444, -4.379448731140335, 4.966558268566445]]
De 4 individuos.


Necesitamos una **función de aptitud** para poder calcular el 'tamaño del pastel'.

Por *razones* (véase el PDF de 'Medida_proba.pdf') -a reserva de que computacionalmente hablando sea demasiado caro- vamos a usar $g(v_i)=e^{-f(v_i)}$ como nuestra función de aptitud.

De este modo, el 'tamaño del pastel' está definido por $F = \sum g(v_i) = \sum e^{-f(v_i)}$.

Para cada $v_i$, $p_i = \frac{e^{-f(v_i)}}{\sum e^{-f(v_i)}} $.

Y para cada $v_i$, su probabilidad acumulativa es $q_i = \sum_{j=i}^{i} \frac{e^{-f(v_j)}}{\sum e^{-f(v_j)}} $.


In [5]:
'''Definimos una función de aptitud que recibe como parámetros una
   función de prueba y una población de 2k individuos y escupe una
   lista con las probabilidades de selección (p_i) de todos los 
   individuos de la población. Además de que nos da al mejor individuo
   en la población.'''

def Aptitudes_Y_Mas_Apto(funcion, poblacion):
    # Generamos una lista con las evaluaciones en la función objetivo
    evaluaciones = [funcion(individuo) for individuo in poblacion]

    #print("Evaluaciones:", [float(eval) for eval in evaluaciones], "\n")

    # Generamos una lista de las aptitudes de los individuos en la 
    # población.
    aptitudes = [np.exp(-evaluacion) for evaluacion in evaluaciones]

    #print("Aptitudes:", [float(apt) for apt in aptitudes], "\n")

    # Encontramos el índice del mejor individuo (menor evaluación)
    mejorIndice = np.argmax(aptitudes)
    masApto = poblacion[mejorIndice]

    #print("El individuo más apto fue:", masApto, "\n")

    # Obtenemos el 'tamaño del pastel':
    sumaAptitudes = sum(aptitudes)

    # Calculamos las probabilidades de selección (p_i) para todos los
    # individuos:
    probabilidades = [(aptitud/sumaAptitudes) for aptitud in aptitudes]

    #print("Probabilidades de selección:", [float(pi) for pi in probabilidades])


    return probabilidades, masApto


In [6]:
'''Veamos un ejemplo de implementación'''

# Consideremos a la función Rastrigin como función de prueba.
def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  suma = 0
  for x in lista:
    suma += x**2 - 10 * np.cos(2* np.pi *x)
  return 10 * n + suma


funcion = Rastrigin
n = 1 # Dimensión en la que están los vectores
k = 2 # Queremos 4 individuos.
intervalo = (-5, 5) # Usaremos el intervalo de la función Rastrigin.
poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"Problación generada:\n{poblacionInicial}\n")
print(f"De {len(poblacionInicial)} individuos.\n")


evaluaciones = [funcion(individuo) for individuo in poblacionInicial]
evaluacionesReales = [float(eval) for eval in evaluaciones]

print(f"Las evaluaciones fueron:\n{evaluacionesReales}\n")

probabilidades, masApto = Aptitudes_Y_Mas_Apto(funcion, poblacionInicial)

probabilidadesReales = [float(pi) for pi in probabilidades]

print(f"El individuo más apto fue {masApto}\n")
print(f"Las probabilidades de selección fueron\n{probabilidadesReales}")


Problación generada:
[[3.152224652563877], [4.618771080247875], [-3.8356609802432584], [1.2082951467400136]]

De 4 individuos.

Las evaluaciones fueron:
[14.172321879576575, 38.67537224260192, 19.58617785560832, 8.869469206048112]

El individuo más apto fue [1.2082951467400136]

Las probabilidades de selección fueron
[0.004952613987617205, 1.1305633885526318e-13, 2.2061076656363306e-05, 0.9950253249356132]


**HASTA AQUí TENEMOS...**

1. Una manera de generar de manera aleatoria una población inicial.
2. La lista de las $p_i$ y el mejor individuo de la población (esto es iterable).

**EL SIGUIENTE PASO ES:**

Generar las probas acumulativas para poder hacer la implementación de la elección de los padres.

In [7]:
'''Definimos una función que recibe la lista de probabilidades 
   de selección y nos escupe la que corresponde a la de las 
   probabilidades acumulativas.'''

def Probas_Acumulativas(probabilidades):

    # Calculamos las probabilidades acumulativas (q_i) para todos los
    # individuos:
    probasAcumulativas = []
    sumaAcumulada = 0

    for proba in probabilidades:
        sumaAcumulada += proba
        probasAcumulativas.append(sumaAcumulada)

    #return probasAcumulativas
    return probasAcumulativas


In [8]:
'''Veamos un ejemplo de implementación'''

# Consideremos a la función Rastrigin como función de prueba.
def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  suma = 0
  for x in lista:
    suma += x**2 - 10 * np.cos(2* np.pi *x)
  return 10 * n + suma


funcion = Rastrigin
n = 1 # Dimensión en la que están los vectores
k = 2 # Queremos 4 individuos.
intervalo = (-5, 5) # Usaremos el intervalo de la función Rastrigin.
poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"Problación generada:\n{poblacionInicial}\n")
print(f"De {len(poblacionInicial)} individuos.\n")


evaluaciones = [funcion(individuo) for individuo in poblacionInicial]
evaluacionesReales = [float(eval) for eval in evaluaciones]

print(f"Las evaluaciones fueron:\n{evaluacionesReales}\n")

probabilidades, masApto = Aptitudes_Y_Mas_Apto(funcion, poblacionInicial)

probabilidadesReales = [float(pi) for pi in probabilidades]

print(f"El individuo más apto fue {masApto}\n")
print(f"Las probabilidades de selección fueron\n{probabilidadesReales}\n")

probasAcumuladas = Probas_Acumulativas(probabilidades)
probasAcumuladasReales = [float(qi) for qi in probasAcumuladas]

print(f"Las probabilidades acumulativas fueron\n{probasAcumuladasReales}\n")


Problación generada:
[[1.4530575560774937], [4.60895833721565], [-1.9136798903608385], [0.1132548234493509]]

De 4 individuos.

Las evaluaciones fueron:
[21.679548641342528, 38.98918320129597, 5.097268463307293, 2.4396537471300173]

El individuo más apto fue [0.1132548234493509]

Las probabilidades de selección fueron
[4.118975920745925e-09, 1.251153168128615e-16, 0.065521228515773, 0.934478767365251]

Las probabilidades acumulativas fueron
[4.118975920745925e-09, 4.118976045861242e-09, 0.06552123263474904, 1.0]



Necesitamos ahora una función que haga la elección de los padres:

Usamos `tol=1e-9` y `math.isclose()` porque las comparaciones de números de punto flotante (números decimales) pueden ser imprecisas, debido a cómo las computadoras los manejan. 

`math.isclose()` permite manejar esta imprecisión al verificar si `r` y `probAcumulada` son prácticamente iguales. Esto evita errores en situaciones donde `r` cae "justo en el filo" que separa las probabilidades acumulativas entre dos individuos.

`tol=1e-9`  define la tolerancia. Si `r` está a una distancia menor que $10^{-9}$ de `probAcumulada`, lo tratamos como si fueran iguales.

En general, en este tipo de algoritmos, cuando $r$ cae exactamente en el límite entre dos intervalos, lo más común es que se elija el individuo cuyo intervalo incluye el valor en el extremo inferior. 

Esto significa que si $r$ es exactamente igual a la probabilidad acumulativa de un individuo, se selecciona el individuo más pequeño, es decir, el que corresponde a ese valor acumulativo. O sea, los intervalos son inclusivos en su límite inferior y exclusivos en su límite superior. Por ejemplo, un intervalo $[0.2, 0.5)$ incluiría 
$r=0.2$, pero no $r=0.5$.

In [22]:
'''Estas funciones son las que nos ayudan a generar a los padres mediante
   el uso de la ruleta.'''
   

'''La primera es para seleccionar padres (uno) y recibe como parámetro 
   la lista de las probabilidades acumulativas.
   
   Nos retornará el índice en la lista que le corresponde al padre 
   elegido con esa regla.'''

import random
import math
import time

def Seleccionar_Padre_Ruleta(probasAcumuladas):
   r = random.random() # Generamos un número aleatorio entre 0 y 1

   tol=1e-9
   for i, probAcumulada in enumerate(probasAcumuladas):
      if r<= probAcumulada or math.isclose(r, probAcumulada, abs_tol=tol):
         return i # Índice del individuo seleccionado.
      


'''Le segunda es para generar la lista de padres elegidos para 
   reproducirse. Necesitamos 2k-1 nuevos individuos, así que se van a 
   elegir 2(2k-1) parejas.
   La función recibe como parámetros: la lista de individuos, las
   probabilidades aculativas y tiempo máximo de ejecución para intentar
   evitar la generación de clones (en este caso, vamos a considerar un máximo
   de 10 segundos para cada elección de un segundo padre, pero tenemos la 
   apertura de modificarlo después).'''
   

def Generar_Parejas(individuos, probasAcumuladas, maxTiempo=5):
   
   parejas = [] # Aquí guardamos a la lista de parejas que nos darán a
                # la siguiente generación.

   parejasNecesarias = 2*(len(individuos)-1)

   clonesGenerados = 0 # Sólo por curiosidad, vamos a llevar un
                       # contador de clones.

   for _ in range(parejasNecesarias):
      
      padre1_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

      tiempoInicio = time.time() # Registramos el tiempo de inicio.

      # En principio, no queremos clones. Se intentará elegir un segundo 
      # padre distinto y una vez que hayamos rebasado el máximo de tiempo
      # nos resignamos y elegimos un clon.
      while True:
         
         padre2_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

         if padre1_idx != padre2_idx: # Comparamos los índices y si 
                                      # tenemos padres distintos, terminamos.
            break
      
      # Si el tiempo transcurrido supera maxTiempo, permitimos un clon
      if time.time() - tiempoInicio > maxTiempo: 
         padre2_idx = padre1_idx
         clonesGenerados +=1 # Agregamos el clon al contador.
         break
      
      # Guardamos la pareja de padres usando sus índices:
      parejas.append((individuos[padre1_idx], individuos[padre2_idx]))

   return parejas, clonesGenerados


In [27]:
'''Veamos un ejemplo de implementación'''

# Consideremos a la función Rastrigin como función de prueba.
def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  suma = 0
  for x in lista:
    suma += x**2 - 10 * np.cos(2* np.pi *x)
  return 10 * n + suma


funcion = Rastrigin
n = 10 # Dimensión en la que están los vectores
k = 60 # Queremos 2k individuos.
intervalo = (-5, 5) # Usaremos el intervalo de la función Rastrigin.
poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"Problación generada:\n{poblacionInicial}\n")
print(f"De {len(poblacionInicial)} individuos.\n")


evaluaciones = [funcion(individuo) for individuo in poblacionInicial]
evaluacionesReales = [float(eval) for eval in evaluaciones]

print(f"Las evaluaciones fueron:\n{evaluacionesReales}\n")

probabilidades, masApto = Aptitudes_Y_Mas_Apto(funcion, poblacionInicial)

probabilidadesReales = [float(pi) for pi in probabilidades]

print(f"El individuo más apto fue {masApto}\n")
print(f"Las probabilidades de selección fueron\n{probabilidadesReales}\n")

probasAcumuladas = Probas_Acumulativas(probabilidades)
probasAcumuladasReales = [float(qi) for qi in probasAcumuladas]

print(f"Las probabilidades acumulativas fueron\n{probasAcumuladasReales}\n")

parejas, numDeClones = Generar_Parejas(poblacionInicial, probasAcumuladas)

print(f"Los padres de la siguiente generación son:\n{parejas}\n")
print(f"Tenemos {len(parejas)} parejas de padres, pues queremos generear {int(len(parejas)/2)} nuevos individuos.\n")
print(f"Se permitirá un total de {numDeClones} clones.\n")


Problación generada:
[[-1.1634723085436791, 2.8596611063878647, -2.1308887742547036, 1.0657521308617284, -2.9713963076398366, -1.4779349663191956, 1.955327821821795, 2.5092999873873945, 0.8696675728833423, 0.9131095136674041], [-1.721497688456446, -0.10070506071095231, 3.1545947254755387, 1.3606864518600705, 3.7537226507576555, 3.945926455955039, -1.835485878910962, 1.6428059630307965, 3.5650079028984845, 0.7218102139727511], [0.15841670818822617, -0.7330004235887886, 1.0125538808399366, -4.704887558293654, -1.7740601038928636, -0.04752563886074146, 4.140453937205919, 2.7257642425398343, -0.40587929423892977, 0.5572641148725754], [1.3706348305186093, 0.2446633728624592, 4.833494596863385, -4.004394086496481, -2.421171439673808, 4.927165245538742, -0.6495409980525846, 0.386326671857927, 3.139949576102266, -4.152180767705511], [-0.6127432481954829, -2.34574524012803, -4.10711734026027, 1.6804539871144932, -3.062633575450997, 2.2536480214222854, -1.3310988397806165, 0.692496662008848, -3.

**HASTA AQUí TENEMOS...**

1. Una manera de generar de manera aleatoria una población inicial.
2. La lista de las $p_i$ y el mejor individuo de la población (esto es iterable).
3. La lista de las $q_i$ (esto es iterable también).
4. Tenemos la elección de los padres por medio de la ruleta (esto también es iterable).

**EL SIGUIENTE PASO ES:**

Codificar las parejas de padres como vectores binarios e implementar los distintos operadores de reproducción usando bits.

In [77]:
'''Funciones auxiliares para la codificación en bianrio.'''

def codifica_real(x, n_bit, intervalo):
    """
    Codifica un número real en el intervalo [a,b] utilizando nBit bits y una partición uniforme en [a, b].

    Parámetros:
    x: número a real a codificar.
    nBit: número de bits a utilizar.
    a: Extremo izquierdo del intervalo.
    b: Extremo derecho del intervalo.

    Retorno:
    Arreglo binario que representa al número real x con nBit bits en el intervalo [a, b].
    """
    
    a, b = intervalo
    
    # Calcula la precisión de la representación.
    precision = (b - a) / (2 ** n_bit)

    # Asegura que el número esté dentro del rango de la partición.
    x = max(a, min(b, x))

    # Calcula el índice del número en la partición.
    index = int((x - a) / precision)

    # Codifica el índice a binario usando nuestro codigo para codificar naturales.
    if index < 0 or index >= (1 << n_bit):
        raise ValueError(f"Índice fuera del rango representable con {n_bit} bits.")

    x_binario = [0] * n_bit
    for i in range(n_bit - 1, -1, -1):
        x_binario[i] = index & 1
        index >>= 1

    # Devuelve el arreglo binario que representa al número real 'x'.
    return x_binario

def codifica_vector(vector_reales, n_bit, intervalo):
    """
    Codifica un vector de números reales en un vector de vectores binarios utilizando nBit bits.

    Parámetros:
    vector_reales: Arreglo de números reales a codificar
    dim_x: Diimensión del vector vector_reales
    nBit: Número de bits a utilizar para las entradas de nuestro arreglo.
    a: Extremo izquierdo del intervalo.
    b: Extremo derecho del intervalo.

    Retorno:
    Arreglo de arreglos arreglos binarios, donde cada subarreglo representa un número real codificado en binario
    """
    
    a, b = intervalo

    dim_x = len(vector_reales)

    # Inicializa un arreglo vacío para almacenar los vectores binarios.
    vector_binario = []

    # Itera sobre cada entrada de nuestro vector de reales.
    for i in range(dim_x):
        numero = vector_reales[i]

        # Codifica cada número utilizando la función codifica
        binario = codifica_real(numero, n_bit, intervalo)

        # Añade el resultado codificado al vector principal
        vector_binario.append(binario)

    #Devuelve un arreglo de arreglos binarios.
    return vector_binario


print(f"El vector {[1.4,1.2,3,1.5674]} en binario con {10} bits es:\n")
print(codifica_vector([1.4,1.2,3,1.5674],10,(1,5)))


El vector [1.4, 1.2, 3, 1.5674] en binario con 10 bits es:

[[0, 0, 0, 1, 1, 0, 0, 1, 1, 0], [0, 0, 0, 0, 1, 1, 0, 0, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 1, 0, 0, 0, 1]]


In [78]:
'''Primero, viene la codificación en vectores binarios del estilo
   [[0,1,1,1,...], ..., [1, 1, 1, 0,...]]'''

def Padres_Binarios(parejas, nBits, intervalo):
   parejasBinarias = []

   for padre1, padre2 in parejas:

      padre1Bin = codifica_vector(padre1, nBits, intervalo)
      padre2Bin = codifica_vector(padre2, nBits, intervalo)

      parejasBinarias.append((padre1Bin, padre2Bin))
   
   return parejasBinarias


In [88]:
'''Veamos un ejemplo de implementación'''

# Consideremos a la función Rastrigin como función de prueba.
def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  suma = 0
  for x in lista:
    suma += x**2 - 10 * np.cos(2* np.pi *x)
  return 10 * n + suma


funcion = Rastrigin
n = 10 # Dimensión en la que están los vectores
k = 60 # Queremos 2k individuos.
intervalo = (-5, 5) # Usaremos el intervalo de la función Rastrigin.
poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"Problación generada:\n{poblacionInicial}\n")
print(f"De {len(poblacionInicial)} individuos.\n")


evaluaciones = [funcion(individuo) for individuo in poblacionInicial]
evaluacionesReales = [float(eval) for eval in evaluaciones]

print(f"Las evaluaciones fueron:\n{evaluacionesReales}\n")

probabilidades, masApto = Aptitudes_Y_Mas_Apto(funcion, poblacionInicial)

probabilidadesReales = [float(pi) for pi in probabilidades]

print(f"El individuo más apto fue {masApto}\n")
print(f"Las probabilidades de selección fueron\n{probabilidadesReales}\n")

probasAcumuladas = Probas_Acumulativas(probabilidades)
probasAcumuladasReales = [float(qi) for qi in probasAcumuladas]

print(f"Las probabilidades acumulativas fueron\n{probasAcumuladasReales}\n")

parejas, clones = Generar_Parejas(poblacionInicial, probasAcumuladas)

print(f"Los padres de la siguiente generación son:\n{parejas}\n")
print(f"Tenemos {len(parejas)} parejas de padres, pues queremos generear {int(len(parejas)/2)} nuevos individuos.\n")
print(f"Se permitirán {clones} clones\n")

nBits = 15 # Vamos a considerar nBits bits
parejasBinarias = Padres_Binarios(parejas, nBits, intervalo)
print(f"Los padres de la siguiente generación (como vectores binarios) son:\n{parejasBinarias}\n")


Problación generada:
[[-1.7005722340718377, 1.3071079666415617, 0.18726347213671168, 1.7832723136416915, -1.5899993176680027, 4.449984895367402, -4.1420627567510495, -3.240294717637223, 3.9660318305340674, 1.4850100090359417], [1.5320161939839743, -4.0300488108877985, -4.136203939349291, 1.5913141609895556, -4.920464299004841, 2.6673389371807694, -3.824087969154528, -4.745839051435041, -0.06625075637138167, 0.6331110275340057], [1.2613190868375792, -3.1696952045338067, -3.262346219378408, -1.668752541987033, 2.482340962902092, 4.708457967235111, 3.0134663554616967, 1.4014981778934281, -3.677684525528818, -4.317232098686622], [-0.8161118254492035, 1.7799163319233244, -1.3765574363480026, 0.6934346991980185, -3.916073704173445, 0.29160437369393755, -1.3241510478064065, -0.8460454078181776, 1.7550116331206054, -0.5297785192640827], [2.513128263260084, -0.5435780613454169, 2.830152434148002, 3.206174961099041, 1.5407503608928232, -1.8217492622035514, -0.1992675071957155, 0.2602854686787390